Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [37]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [38]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [39]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [40]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [31]:
#To not use GPU, on macbook, GPU memory of 512 MB is too little for most apps.
config = tf.ConfigProto(
          device_count = {'GPU': 0} )

image_size = 28
num_labels = 10

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000

beta=0.06

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  print(tf_train_dataset.shape)
  tf_train_labels = tf.constant(train_labels[:train_subset])
  print(tf_train_labels.shape)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  print(type(weights))
  biases = tf.Variable(tf.zeros([num_labels]))
  print(type(biases))
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss. 
  #Add regularization term
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 801
#num_steps = 201

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph,config=config) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

(1000, 784)
(1000, 10)
<class 'tensorflow.python.ops.variables.Variable'>
<class 'tensorflow.python.ops.variables.Variable'>
Initialized
Loss at step 0: 202.433014
Training accuracy: 8.3%
Validation accuracy: 9.8%
Loss at step 100: 1.184321
Training accuracy: 86.7%
Validation accuracy: 79.5%
Loss at step 200: 0.811755
Training accuracy: 86.6%
Validation accuracy: 79.8%
Loss at step 300: 0.809591
Training accuracy: 86.8%
Validation accuracy: 79.8%
Loss at step 400: 0.809058
Training accuracy: 86.9%
Validation accuracy: 79.8%
Loss at step 500: 0.808828
Training accuracy: 86.9%
Validation accuracy: 79.8%
Loss at step 600: 0.808721
Training accuracy: 87.0%
Validation accuracy: 79.8%
Loss at step 700: 0.808670
Training accuracy: 87.0%
Validation accuracy: 79.8%
Loss at step 800: 0.808644
Training accuracy: 87.0%
Validation accuracy: 79.8%
Test accuracy: 86.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

With num_steps = 200 and batch size of 100, achieve a training accuracy of 100%, but validation = 69% and test accuracy = 76%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
#To not use GPU, on macbook, GPU memory of 512 MB is too little for most apps.
config = tf.ConfigProto(
          device_count = {'GPU': 0} )

image_size = 28
num_labels = 10
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000
hidden_nodes_layer1 = 1024
keep_probab = 0.1
beta = 0.2

def compute_logits(dataset,h_wts,h_bias,o_wts,o_bias,keep_probab=1):
    hidden_layer_linear = tf.matmul(dataset, h_wts) + h_bias
    hidden_layer_out = tf.nn.relu(hidden_layer_linear)
    logits = tf.matmul(tf.nn.dropout(hidden_layer_out,keep_probab), o_wts) + o_bias
    return logits

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  print(tf_train_dataset.shape)
  tf_train_labels = tf.constant(train_labels[:train_subset])
  print(tf_train_labels.shape)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  h_weights_layer1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer1]))

  h_biases_layer1 = tf.Variable(tf.zeros([hidden_nodes_layer1]))
  
  o_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer1, num_labels]))

  o_biases = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = compute_logits(tf_train_dataset, h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases,keep_probab)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
     + beta*(tf.nn.l2_loss(h_weights_layer1) + tf.nn.l2_loss(o_weights))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax( compute_logits(tf_valid_dataset,h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases) )
    
  test_prediction = tf.nn.softmax( compute_logits(tf_test_dataset,h_weights_layer1, h_biases_layer1,
                          o_weights,o_biases) )

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph,config=config) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))



(1000, 784)
(1000, 10)
Initialized
Loss at step 0: 1136.171753
Training accuracy: 10.5%
Validation accuracy: 36.2%
Loss at step 100: 14.298226
Training accuracy: 83.4%
Validation accuracy: 80.0%
Loss at step 200: 6.776614
Training accuracy: 87.7%
Validation accuracy: 80.0%
Loss at step 300: 3.164962
Training accuracy: 90.0%
Validation accuracy: 79.8%
Loss at step 400: 2.655899
Training accuracy: 89.1%
Validation accuracy: 79.7%
Loss at step 500: 2.638416
Training accuracy: 91.4%
Validation accuracy: 79.9%
Loss at step 600: 1.088229
Training accuracy: 92.3%
Validation accuracy: 79.9%
Loss at step 700: 1.829051
Training accuracy: 92.6%
Validation accuracy: 79.9%
Loss at step 800: 1.605844
Training accuracy: 93.4%
Validation accuracy: 79.9%
Test accuracy: 86.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [52]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100000
hidden_nodes_layer1 = 500
hidden_nodes_layer2 = 500
#learning_rate = 0.1
batch_size = 345

starter_learning_rate = 0.5
keep_probab = 0.5
beta = 0.003

def compute_logits_2layer(dataset,h1_wts,h1_bias,h2_wts,h2_bias,o_wts,o_bias,keep_probab=1.0):
    h1_linear = tf.matmul(dataset, h1_wts) + h1_bias
    h1_out = tf.nn.relu(h1_linear)
    h2_linear = tf.matmul(tf.nn.dropout(h1_out,keep_probab),h2_wts) + h2_bias
    h2_out = tf.nn.relu(h2_linear)
    logits = tf.matmul(tf.nn.dropout(h2_out,keep_probab), o_wts) + o_bias
    return logits

graph = tf.Graph()
with graph.as_default():

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.96, staircase=True)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  h_weights_layer1 = tf.Variable(
    tf.random_normal(shape=[image_size * image_size, hidden_nodes_layer1],stddev=0.01))

  h_biases_layer1 = tf.Variable(tf.zeros([hidden_nodes_layer1]))

  h_weights_layer2 = tf.Variable(
    tf.random_normal(shape=[hidden_nodes_layer1, hidden_nodes_layer2],stddev=0.01))

  h_biases_layer2 = tf.Variable(tf.zeros([hidden_nodes_layer2]))
  
  o_weights = tf.Variable(
    tf.random_normal(shape=[hidden_nodes_layer2, num_labels],stddev=0.01))

  o_biases = tf.Variable(tf.zeros([num_labels]))
    
  tf.summary.histogram("h_wts_layer1",h_weights_layer1)
  tf.summary.histogram("h_bias_layer1",h_biases_layer1)
  tf.summary.histogram("h_wts_layer2",h_weights_layer2)
  tf.summary.histogram("h_bias_layer2",h_biases_layer2)
  tf.summary.histogram("o_wts",o_weights)
  tf.summary.histogram("o_bias",o_biases)

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = compute_logits_2layer(tf_train_dataset, h_weights_layer1, h_biases_layer1,h_weights_layer2, h_biases_layer2,
                          o_weights,o_biases,keep_probab)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + beta*(tf.nn.l2_loss(h_weights_layer1)+
                                                                              tf.nn.l2_loss(h_weights_layer2) +
                                                                              tf.nn.l2_loss(o_weights))

  tf.summary.scalar('loss',loss)

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  #
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


  #optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax( compute_logits_2layer(tf_valid_dataset,h_weights_layer1, h_biases_layer1,h_weights_layer2, h_biases_layer2,
                          o_weights,o_biases) )
    
  test_prediction = tf.nn.softmax( compute_logits_2layer(tf_test_dataset,h_weights_layer1, h_biases_layer1,h_weights_layer2, h_biases_layer2,
                          o_weights,o_biases) )


num_steps = 10001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph,config=config) as session:
    
  writer = tf.summary.FileWriter("./logs/",session.graph)
  merged = tf.summary.merge_all()
    
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
    
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions,summary = session.run(
      [optimizer, loss, train_prediction , merged], feed_dict=feed_dict)
    
    writer.add_summary(summary, step)  # Write summary
        
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.399873
Minibatch accuracy: 8.4%
Validation accuracy: 10.0%
Test accuracy: 10.0%
Minibatch loss at step 500: 0.633596
Minibatch accuracy: 84.3%
Validation accuracy: 85.4%
Test accuracy: 91.2%
Minibatch loss at step 1000: 0.669943
Minibatch accuracy: 83.2%
Validation accuracy: 86.3%
Test accuracy: 92.0%
Minibatch loss at step 1500: 0.503212
Minibatch accuracy: 90.1%
Validation accuracy: 87.1%
Test accuracy: 92.4%
Minibatch loss at step 2000: 0.567106
Minibatch accuracy: 87.0%
Validation accuracy: 87.6%
Test accuracy: 93.0%
Minibatch loss at step 2500: 0.486662
Minibatch accuracy: 88.1%
Validation accuracy: 87.7%
Test accuracy: 93.1%
Minibatch loss at step 3000: 0.583832
Minibatch accuracy: 87.2%
Validation accuracy: 87.6%
Test accuracy: 93.1%
Minibatch loss at step 3500: 0.549214
Minibatch accuracy: 87.0%
Validation accuracy: 88.1%
Test accuracy: 93.5%
Minibatch loss at step 4000: 0.555434
Minibatch accuracy: 87.8%
Validation accuracy: 88.1%
Test a